In [12]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
import pandas as pd
from tqdm import tqdm
from backbone import Backbone
from tensorflow import keras
from PIL import Image

In [13]:
model = Backbone((240, 240))
model.build([89])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 240, 240, 3) 0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 240, 240, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, 240, 240, 3)  7           rescaling_1[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 241, 241, 3)  0           normalization_1[0][0]            
____________________________________________________________________________________________

In [14]:
latest = tf.train.latest_checkpoint("checkpoints/chckp_89_16_07")
model.load_weights(latest)

In [15]:
dataset = pd.read_csv("dataset/test/test_flickr.csv")
dataset


,img_path,lvl_2
0,4298386222_a31d91b5c6.jpg,45
1,12294843445_7dfa214cf9.jpg,47
2,3501789921_f198cb36cb.jpg,67
3,24474377831_b8bea2ae78.jpg,30
4,3123270263_70c94b8b15.jpg,42
...,...,...
9995,8566150188_4dd008391c.jpg,20
9996,33460315358_2327647073.jpg,44
9997,10113610_4983a3a8c2.jpg,16
9998,10488073104_b29011481c.jpg,9


In [18]:
from tqdm import tqdm 
BASE_PATH = "dataset/crawl/flickr/test"
results = []
top5 = 0
top10 = 0
top1 = 0
with tqdm(total=dataset.shape[0]) as pbar:
    for _,row in dataset.iterrows():
        pbar.update(1)
        path = os.path.join(BASE_PATH, row["img_path"])

        background = Image.open(path).convert('RGB')
        background = background.resize((240, 240))
        

        img_array_p = np.array(background)
        img_array_p = np.expand_dims(img_array_p, 0)
        res = model.get_model().predict(img_array_p)
        
        results.append(
        {
            "img_path": row["img_path"],
            "true_class": row["lvl_2"],
            "predicted_class": res.argmax(),
            "confidence": res.max()
        })
        pred_top5 = res[0].argsort()[::-1][:5]
        pred_top10 = res[0].argsort()[::-1][:10]
        if(row["lvl_2"] == res.argmax()):
            top1 +=1
        if(row["lvl_2"] in pred_top5):
            #print(pred_top5)
            top5 +=1
        if(row["lvl_2"] in pred_top10):
            top10 +=1
        
        


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [2:18:08<00:00,  1.21it/s]


In [19]:
output = pd.DataFrame(results)
output.head(100)
(output["true_class"] == output["predicted_class"]).sum() / output.shape[0]


0.1431

In [20]:
accuracy = (output["true_class"] == output["predicted_class"]).sum() / output.shape[0]
print("Top1: " + str(accuracy * 100) +"%")
print("Top5: " + str(top5 / output.shape[0] * 100) +"%")
print("Top10: " + str(top10 / output.shape[0] * 100) +"%")

print("Improvement over the random guess (IOR) " + str(accuracy * model.get_output_classes()))

Top1: 14.31
Top5: 32.79
Top10: 45.95


In [21]:
output.to_csv("results/res_89.csv", index=False)